In [173]:
import json

In [ ]:
import json

results_file = "Qwen3_results/BFCL_v3_simple_Qwen3_results.json"
result_obj = []

try:
    with open(results_file, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            # 跳过空行
            if line.strip():
                try:
                    result_obj.append(json.loads(line))
                except json.JSONDecodeError:
                    print(f"警告: 第 {i+1} 行不是有效的JSON，已跳过。")
except FileNotFoundError:
    print(f"错误: 文件 '{results_file}' 未找到。请检查文件路径。")

# print(len(result_obj)) # 可以打印一下看看成功加载了多少个对象

In [3]:
result_obj[14]["conversations"][0]["turns"][0]["success"]

False

In [4]:
result_obj[14]

{'scenario': 'simple_14',
 'conversations': [{'id': 'simple_14_conversation',
   'turns': [{'query': 'Calculate the derivative of the function 3x^2 + 2x - 1.',
     'reference_response': '',
     'actual_response': 'The derivative of the function 3x^2 + 2x - 1 is 6x + 2. \n\nNo further operations are needed since we\'ve successfully calculated the derivative. The "Success" output indicates the operation completed correctly, though it didn\'t display the derivative expression directly. The derivative you\'re looking for is 6x + 2.',
     'expected_calls': [{'name': 'calculate_derivative',
       'required': True,
       'arguments': [{'name': 'function',
         'value': ['3x**2 + 2x - 1', 'lambda x: 3x**2 + 2x - 1'],
         'type': 'str',
         'required': True},
        {'name': 'x_value',
         'value': ['', 0.0],
         'type': 'float',
         'required': False}]}],
     'actual_calls': [{'function': 'calculate_derivative',
       'arguments': {'function': '3x^2 + 2x - 

In [176]:
count = 0
faild_list = []
error = 0
for i in range(len(result_obj)):
    try:
        if result_obj[i]["conversations"][0]["turns"][0]["success"] == False:
            count+=1
            faild_list.append(i)
    except:
        error+=1

In [177]:
error

0

In [178]:
count

63

In [180]:
i = 10
print(result_obj[faild_list[i]]["scenario"])
print(result_obj[faild_list[i]]["conversations"][0]["turns"][0]["validation_errors"])

simple_89
["Call 1, Arg 'conditions': actual_value '{'school': 'Bluebird High School', 'major': 'Science'}' is not in expected value list", "Call 1, Arg 'conditions': actual_value '{'school': 'Bluebird High School', 'major': 'Science'}' type is not matching expected_type 'Dict[str, str]' "]


In [181]:
result_obj[faild_list[i]]

{'scenario': 'simple_89',
 'conversations': [{'id': 'simple_89_conversation',
   'turns': [{'query': "Fetch all records for students studying Science in 'Bluebird High School' from the StudentDB.",
     'reference_response': '',
     'actual_response': 'The database query returned no records for Science students at Bluebird High School. This could mean either: (1) there are no Science students currently enrolled at Bluebird High School in the database, or (2) the database/table name or conditions need to be adjusted.\n\nWould you like me to try any of these alternatives?\n1. Verify the database and table names are correct\n2. Check for students in related science fields (e.g., "Biology", "Chemistry")\n3. Look for slight variations in the school name (e.g., "Blue Bird High School")\n\nPlease let me know how you\'d like to proceed.',
     'expected_calls': [{'name': 'db_fetch_records',
       'required': True,
       'arguments': [{'name': 'database_name',
         'value': ['StudentDB']

In [ ]:
import json

def load_json_stream(file_path):
    """
    讀取一個包含多個連續JSON對象的文件。
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            file_content = f.read().replace('}\n{', '},{')
        json_array_string = f"[{file_content}]"
        return json.loads(json_array_string)
    except Exception:
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                return [json.loads(line) for line in f if line.strip()]
        except json.JSONDecodeError as e:
            print(f"解析文件 '{file_path}' 時出錯: {e}")
            return None

def analyze_scenarios_by_id(file1_path, file2_path):
    """
    以 scenario ID 為基準進行匹配和分析，並記錄僅存在於單一文件中的案例ID。
    """
    data1 = load_json_stream(file1_path)
    data2 = load_json_stream(file2_path)

    if data1 is None or data2 is None:
        return None

    scenarios_f1 = {s['scenario']: s for s in data1}
    scenarios_f2 = {s['scenario']: s for s in data2}

    all_scenario_ids = set(scenarios_f1.keys()) | set(scenarios_f2.keys())
    
    # 初始化計數器和ID列表
    success_f1 = 0
    success_f2 = 0
    both_failed = 0
    only_in_f1_ids = []
    only_in_f2_ids = []

    for sid in all_scenario_ids:
        scenario1 = scenarios_f1.get(sid)
        scenario2 = scenarios_f2.get(sid)

        in_f1 = scenario1 is not None
        in_f2 = scenario2 is not None

        is_s1_success = False
        if in_f1:
            is_s1_success = all(not turn.get('validation_errors') 
                                for conv in scenario1.get('conversations', []) 
                                for turn in conv.get('turns', []))
            if is_s1_success:
                success_f1 += 1

        is_s2_success = False
        if in_f2:
            is_s2_success = all(not turn.get('validation_errors') 
                                for conv in scenario2.get('conversations', []) 
                                for turn in conv.get('turns', []))
            if is_s2_success:
                success_f2 += 1
        
        # --- 更新統計 ---
        if in_f1 and not in_f2:
            only_in_f1_ids.append(sid) # 收集ID
        elif not in_f1 and in_f2:
            only_in_f2_ids.append(sid) # 收集ID
        elif in_f1 and in_f2:
            if not is_s1_success and not is_s2_success:
                both_failed += 1

    return {
        "file1_path": file1_path,
        "file2_path": file2_path,
        "file1_total": len(scenarios_f1),
        "file2_total": len(scenarios_f2),
        "file1_success_count": success_f1,
        "file2_success_count": success_f2,
        "both_failed_count": both_failed,
        "total_unique_scenarios": len(all_scenario_ids),
        # 返回排序後的ID列表
        "only_in_file1_ids": sorted(only_in_f1_ids),
        "only_in_file2_ids": sorted(only_in_f2_ids),
    }

# --- 主程序 ---
file_pairs = {
    "第一組 (Multiple)": ("Qwen3_results/BFCL_v3_multiple_Qwen3_results_llamaindex.json", "Qwen3_results/BFCL_v3_multiple_Qwen3_results.json"),
    "第二組 (Parallel)": ("Qwen3_results/BFCL_v3_parallel_Qwen3_results_llamaindex.json", "Qwen3_results/BFCL_v3_parallel_Qwen3_results.json"),
    "第三組 (Parallel Multiple)": ("Qwen3_results/BFCL_v3_parallel_multiple_Qwen3_results_llamaindex.json", "Qwen3_results/BFCL_v3_parallel_multiple_Qwen3_results.json"),
    "第四組 (Simple)": ("Qwen3_results/BFCL_v3_simple_Qwen3_results_llamaindex.json", "Qwen3_results/BFCL_v3_simple_Qwen3_results.json")
}

# 遍歷每一對文件並打印新的分析結果
for group_name, (file1, file2) in file_pairs.items():
    results = analyze_scenarios_by_id(file1, file2)
    if results:
        print("---")
        print(f"**{group_name}**")
        print(f"總計有 {results['total_unique_scenarios']} 個獨立案例。")
        print(f"* 文件 '{results['file1_path']}':")
        print(f"    * 成功案例數: {results['file1_success_count']} / {results['file1_total']}")
        print(f"* 文件 '{results['file2_path']}':")
        print(f"    * 成功案例數: {results['file2_success_count']} / {results['file2_total']}")
        print(f"* **對比**: ")
        print(f"    * 兩邊都失敗的案例: {results['both_failed_count']}")
        
        # 新增邏輯：如果列表不為空，則打印數量和具體案例名
        if results['only_in_file1_ids']:
            print(f"    * 僅存在於文件1的案例: {len(results['only_in_file1_ids'])}")
            print(f"        * 案例名: {results['only_in_file1_ids']}")
        if results['only_in_file2_ids']:
            print(f"    * 僅存在於文件2的案例: {len(results['only_in_file2_ids'])}")
            print(f"        * 案例名: {results['only_in_file2_ids']}")
        print("---\n")

---
**第一組 (Multiple)**
總計有 200 個獨立案例。
* 文件 'deepseek-v3-0324_results/BFCL_v3_multiple_deepseek-v3-0324_results_llamaindex.json':
    * 成功案例數: 188 / 198
* 文件 'deepseek-v3-0324_results/BFCL_v3_multiple_deepseek-v3-0324_results.json':
    * 成功案例數: 189 / 200
* **對比**: 
    * 兩邊都失敗的案例: 8
    * 僅存在於文件2的案例: 2
        * 案例名: ['multiple_143', 'multiple_197']
---

---
**第二組 (Parallel)**
總計有 200 個獨立案例。
* 文件 'deepseek-v3-0324_results/BFCL_v3_parallel_deepseek-v3-0324_results_llamaindex.json':
    * 成功案例數: 171 / 200
* 文件 'deepseek-v3-0324_results/BFCL_v3_parallel_deepseek-v3-0324_results.json':
    * 成功案例數: 171 / 200
* **對比**: 
    * 兩邊都失敗的案例: 20
---

---
**第三組 (Parallel Multiple)**
總計有 200 個獨立案例。
* 文件 'deepseek-v3-0324_results/BFCL_v3_parallel_multiple_deepseek-v3-0324_results_llamaindex.json':
    * 成功案例數: 171 / 198
* 文件 'deepseek-v3-0324_results/BFCL_v3_parallel_multiple_deepseek-v3-0324_results.json':
    * 成功案例數: 175 / 200
* **對比**: 
    * 兩邊都失敗的案例: 17
    * 僅存在於文件2的案例: 2
        * 案例名: ['para

In [4]:
import json

def load_json_stream(file_path):
    """
    讀取一個包含多個連續JSON對象的文件。
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            file_content = f.read().replace('}\n{', '},{')
        json_array_string = f"[{file_content}]"
        return json.loads(json_array_string)
    except Exception:
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                return [json.loads(line) for line in f if line.strip()]
        except json.JSONDecodeError as e:
            print(f"解析文件 '{file_path}' 時出錯: {e}")
            return None

def analyze_scenarios_by_id(file1_path, file2_path):
    """
    以 scenario ID 為基準進行匹配和分析，並記錄僅存在於單一文件中的案例ID。
    """
    data1 = load_json_stream(file1_path)
    data2 = load_json_stream(file2_path)

    if data1 is None or data2 is None:
        return None

    scenarios_f1 = {s['scenario']: s for s in data1}
    scenarios_f2 = {s['scenario']: s for s in data2}

    all_scenario_ids = set(scenarios_f1.keys()) | set(scenarios_f2.keys())
    
    # 初始化計數器和ID列表
    success_f1 = 0
    success_f2 = 0
    both_failed = 0
    only_in_f1_ids = []
    only_in_f2_ids = []

    for sid in all_scenario_ids:
        scenario1 = scenarios_f1.get(sid)
        scenario2 = scenarios_f2.get(sid)

        in_f1 = scenario1 is not None
        in_f2 = scenario2 is not None

        is_s1_success = False
        if in_f1:
            is_s1_success = all(not turn.get('validation_errors') 
                                for conv in scenario1.get('conversations', []) 
                                for turn in conv.get('turns', []))
            if is_s1_success:
                success_f1 += 1

        is_s2_success = False
        if in_f2:
            is_s2_success = all(not turn.get('validation_errors') 
                                for conv in scenario2.get('conversations', []) 
                                for turn in conv.get('turns', []))
            if is_s2_success:
                success_f2 += 1
        
        # --- 更新統計 ---
        if in_f1 and not in_f2:
            only_in_f1_ids.append(sid) # 收集ID
        elif not in_f1 and in_f2:
            only_in_f2_ids.append(sid) # 收集ID
        elif in_f1 and in_f2:
            if not is_s1_success and not is_s2_success:
                both_failed += 1

    return {
        "file1_path": file1_path,
        "file2_path": file2_path,
        "file1_total": len(scenarios_f1),
        "file2_total": len(scenarios_f2),
        "file1_success_count": success_f1,
        "file2_success_count": success_f2,
        "both_failed_count": both_failed,
        "total_unique_scenarios": len(all_scenario_ids),
        # 返回排序後的ID列表
        "only_in_file1_ids": sorted(only_in_f1_ids),
        "only_in_file2_ids": sorted(only_in_f2_ids),
    }

# --- 主程序 ---
file_pairs = {
    "第一組 (Multiple)": ("Qwen3_results/BFCL_v3_multiple_Qwen3_results_llamaindex.json", "Qwen3_results/BFCL_v3_multiple_Qwen3_results.json"),
    "第二組 (Parallel)": ("Qwen3_results/BFCL_v3_parallel_Qwen3_results_llamaindex.json", "Qwen3_results/BFCL_v3_parallel_Qwen3_results.json"),
    "第三組 (Parallel Multiple)": ("Qwen3_results/BFCL_v3_parallel_multiple_Qwen3_results_llamaindex.json", "Qwen3_results/BFCL_v3_parallel_multiple_Qwen3_results.json"),
    "第四組 (Simple)": ("Qwen3_results/BFCL_v3_simple_Qwen3_results_llamaindex.json", "Qwen3_results/BFCL_v3_simple_Qwen3_results.json")
}

# 遍歷每一對文件並打印新的分析結果
for group_name, (file1, file2) in file_pairs.items():
    results = analyze_scenarios_by_id(file1, file2)
    if results:
        print("---")
        print(f"**{group_name}**")
        print(f"總計有 {results['total_unique_scenarios']} 個獨立案例。")
        print(f"* 文件 '{results['file1_path']}':")
        print(f"    * 成功案例數: {results['file1_success_count']} / {results['file1_total']}")
        print(f"* 文件 '{results['file2_path']}':")
        print(f"    * 成功案例數: {results['file2_success_count']} / {results['file2_total']}")
        print(f"* **對比**: ")
        print(f"    * 兩邊都失敗的案例: {results['both_failed_count']}")
        
        # 新增邏輯：如果列表不為空，則打印數量和具體案例名
        if results['only_in_file1_ids']:
            print(f"    * 僅存在於文件1的案例: {len(results['only_in_file1_ids'])}")
            print(f"        * 案例名: {results['only_in_file1_ids']}")
        if results['only_in_file2_ids']:
            print(f"    * 僅存在於文件2的案例: {len(results['only_in_file2_ids'])}")
            print(f"        * 案例名: {results['only_in_file2_ids']}")
        print("---\n")

---
**第一組 (Multiple)**
總計有 200 個獨立案例。
* 文件 'Qwen3_results/BFCL_v3_multiple_Qwen3_results_llamaindex.json':
    * 成功案例數: 188 / 198
* 文件 'Qwen3_results/BFCL_v3_multiple_Qwen3_results.json':
    * 成功案例數: 190 / 200
* **對比**: 
    * 兩邊都失敗的案例: 9
    * 僅存在於文件2的案例: 2
        * 案例名: ['multiple_143', 'multiple_197']
---

---
**第二組 (Parallel)**
總計有 200 個獨立案例。
* 文件 'Qwen3_results/BFCL_v3_parallel_Qwen3_results_llamaindex.json':
    * 成功案例數: 178 / 200
* 文件 'Qwen3_results/BFCL_v3_parallel_Qwen3_results.json':
    * 成功案例數: 166 / 200
* **對比**: 
    * 兩邊都失敗的案例: 19
---

---
**第三組 (Parallel Multiple)**
總計有 200 個獨立案例。
* 文件 'Qwen3_results/BFCL_v3_parallel_multiple_Qwen3_results_llamaindex.json':
    * 成功案例數: 178 / 198
* 文件 'Qwen3_results/BFCL_v3_parallel_multiple_Qwen3_results.json':
    * 成功案例數: 177 / 200
* **對比**: 
    * 兩邊都失敗的案例: 16
    * 僅存在於文件2的案例: 2
        * 案例名: ['parallel_multiple_15', 'parallel_multiple_9']
---

---
**第四組 (Simple)**
總計有 400 個獨立案例。
* 文件 'Qwen3_results/BFCL_v3_simple_Qwen3_results_